In [ ]:
import re 
import tokenize

# Grammar

A <u>Grammar</u> is defined as G = (N, E, P, S) where:
    
    N = set of non-terminals
    E = set of terminals
    P = set of productions
    S = starting symbol 



In [ ]:
class Grammar:
    @staticmethod
    def parseLine(line):
        equalPos = line.index('=')
        rhs = line[equalPos + 1:].strip('\n').strip(' ')[1:-1]
        return [symbol.strip() for symbol in rhs.split(',')]
    
    @staticmethod
    def fromFile(fileName):
        with open(fileName) as file: 
            N = Grammar.parseLine(file.readline())
            E = Grammar.parseLine(file.readline())
            S = file.readline().split('=')[1].strip()
            P = Grammar.parseRules([line.strip('\n').strip(' ').strip(',') for line in file][1:-1])
            
            return Grammar(N, E, P, S)

    @staticmethod        
    def parseRules(rules):
        result = []
        
        for rule in rules:
            lhs, rhs = rule.split('->')
            lhs = lhs.strip()
            rhs = [ value.strip() for value in rhs.split('|')]
            
            for value in rhs: 
                result.append((lhs, value.split()))
        
        return result 
   
    def __init__(self, N, E, P, S):
        self.N = N 
        self.E = E
        self.P = P
        self.S = S
        
    def isNonTerminal(self, value):
        return value in self.N
    
    def isTerminal(self, value):
        return value in self.E 
    
    def isRegular(self):
        usedInRhs = dict() 
        notAllowedInRhs = list() 
        
        for rule in self.P: 
            lhs, rhs = rule
            hasTerminal = False 
            hasNonTerminal = False
            for char in rhs: 
                if self.isNonTerminal(char): 
                    usedInRhs[char] = True
                    hasNonTerminal = True
                elif self.isTerminal(char): 
                    if hasNonTerminal: 
                        return False
                    hasTerminal = True 
                if char == 'E': 
                    notAllowedInRhs.append(lhs)
                    
            if hasNonTerminal and not hasTerminal: 
                return False
        
        for char in notAllowedInRhs: 
            if char in usedInRhs: 
                return False 
            
        return True
   
    def getProductionsFor(self, nonTerminal): 
        if not self.isNonTerminal(nonTerminal):
            raise Exception('Can only show productions for non-terminals')
            
        return [ prod for prod in self.P if prod[0] == nonTerminal ]
    
    def showProductionsFor(self, nonTerminal):
        productions = self.getProductionsFor(nonTerminal)
        
        print(', '.join([' -> '.join(prod) for prod in productions]))
        
    def __str__(self):
        return 'N = { ' + ', '.join(self.N) + ' }\n' \
             + 'E = { ' + ', '.join(self.E) + ' }\n' \
             + 'P = { ' + ', '.join([' -> '.join([prod[0], ' '.join(prod[1])]) for prod in self.P]) + ' }\n' \
             + 'S = ' + str(self.S) + '\n'

# LR(0) Parser


For parsing the method from the previous labs was used. Tokenizing the input and then obtaining the resulting PIF.
PIF conversion is done based on a codification tables with each non-terminal being map to a unique id, together with the
codes found in the symbol and constant tables. The grammar implementation and code is also the one used in the previous
work, also including regular grammar verification.
The PIF is then used in the LR(0) parsing algorithm as detailed in the course slides as follows:


Closure Computation 

The closure of a set of LR(0) items can be computed using a simple (but important) little algorithm
Algorithm to compute closure(A)
	set A' equal to A.
	while there is some [ N -> beta1 . M beta2 ] exists in A' such that
	M->beta3 exists in P and [ M ->.beta3 ] exists in A' add [M ->.beta3 ] to A'. 

	
The definition of the LR(0) machine is:

Let the set of states of the machine be the set of all sets of LR(0) items for G.
Let the set of final states be all states except the state corresponding to the empty set of LR(0) item.
Let the initial state be the state corresponding to the set

	closure ( {[ S' ->.S$]} ) 

Let the transition function f: pi(x)(Vn U Vt) -> pi be defined by:

	f(pi,x) = closure(goto(pi,x)) 


Reguli tabel LR(0)

    1.dacă [A → α.β] ∈si atunci acțiune(si)=shift
	2.dacă [A → β.] ∈si și A ≠Sʹ atunci acțiune(si)=reduce l, unde l -numărul producției A → β
	3.dacă [Sʹ → S.] ∈si atunci acțiune(si)=acc
	4.dacă goto(si, X) = sj atunci goto(si, X) = sj
	5.toate celelalte valori = eroare


In [ ]:
class Lr0Parser: 
    
    def __init__(self, grammar): 
        self.__grammar = grammar
        self.__workingStack = []
        self.__inputStack = []
        self.__output = [] 
        
    def closure(self, productions): 
        
        if productions == []:
            return None
        C = productions
        finished = False 
        
        while not finished:
            finished = True 
            for dottedProd in C:
                dotIndex = dottedProd[1].index('.')
                alpha = dottedProd[1][:dotIndex]
                Bbeta = dottedProd[1][dotIndex + 1:]

                if len(Bbeta) == 0: 
                    continue
                    
                B = Bbeta[0]
                if self.__grammar.isTerminal(B): 
                    continue
                    
                for prod in self.__grammar.getProductionsFor(B):
                    dottedProd = (prod[0], ['.'] + prod[1])
                    if dottedProd not in C: 
                        C += [ dottedProd ]
                        finished = False 
        return C
        
    
    def goTo(self, state, symbol):
        C = []
        
        for dottedProd in state: 
            dotIndex = dottedProd[1].index('.')
            alpha = dottedProd[1][:dotIndex]
            Xbeta = dottedProd[1][dotIndex + 1:]
            
            if len(Xbeta) == 0: 
                continue
            
            X, beta = Xbeta[0], Xbeta[1:]
            
            if X == symbol:
                resultProd = (dottedProd[0], alpha + [X] + ['.'] + beta)
                C = C + [ resultProd ]
        
        return self.closure(C)
        
    def getCannonicalCollection(self): 
        C = [ self.closure([('S1', ['.', self.__grammar.S])]) ]
        
        finished = False 
        
        while not finished: 
            finished = True 
            
            for state in C: 
                for symbol in self.__grammar.N + self.__grammar.E: 
                    nextState = self.goTo(state, symbol)
                    if nextState is not None and nextState not in C: 
                        C = C + [ nextState ]
                        finished = False
    
        return C
    
    def getTable(self): 
        states = self.getCannonicalCollection()
        table = [{} for _ in range(len(states))]
        
        for index, state in enumerate(states):
            meetsFirstRule = 0
            meetsSecondRule = 0 
            meetsThirdRule = 0 
            
            for prod in state: 
                dotIndex = prod[1].index('.')
                alpha = prod[1][:dotIndex]
                beta = prod[1][dotIndex + 1:]
                if len(beta) != 0:
                    meetsFirstRule += 1
                else:
                    if prod[0] != 'S1':
                        meetsSecondRule += 1
                        productionIndex = self.__grammar.P.index((prod[0], alpha))
                    elif alpha == [self.__grammar.S]: 
                        meetsThirdRule += 1
                
            if meetsFirstRule == len(state): 
                table[index]['action'] = 'shift'
                
            elif meetsSecondRule == len(state):
                table[index]['action'] = 'reduce ' + str(productionIndex)
                
            elif meetsThirdRule == len(state):
                table[index]['action'] = 'acc'
                
            else: 
                raise(Exception('No action detected for state ' + str(index) + ' ' + str(state)))
                
            
            for symbol in self.__grammar.N + self.__grammar.E: 
                nextState = self.goTo(state, symbol)
                if nextState in states: 
                    table[index][symbol] = states.index(nextState)
            
        return table
    
    
    def parse(self, inputSequence): 
        table = self.getTable()
        
        self.__workingStack = ['0']
        self.__inputStack = [symbol for symbol in inputSequence]
        self.__output = []
        
        while len(self.__workingStack) != 0: 
            state = int(self.__workingStack[-1])
            if len(self.__inputStack) > 0:
                symbol = self.__inputStack.pop(0)
            else: 
                symbol = None
                
            if table[state]['action'] == 'shift': 
                if symbol not in table[state]: 
                    print('workstack', self.__workingStack)
                    print('inputstack', self.__inputStack)
                    print('state', state)
                    print('symbol', symbol)
                    
                    raise(Exception('Cannot parse shift'))
                self.__workingStack.append(symbol)
                self.__workingStack.append(table[state][symbol])
                
            elif table[state]['action'] == 'acc': 
                if len(self.__inputStack) != 0:
                    raise(Exception('Cannot Parse acc'))
                
                self.__workingStack.clear()
                
            else: 
                reducedState = int(table[state]['action'].split(' ')[1])
                reducedProduction = self.__grammar.P[reducedState]
                
                toRemoveFromWorkingStack = [symbol for symbol in reducedProduction[1]]
                
                while len(toRemoveFromWorkingStack) > 0 and len(self.__workingStack) > 0:
                    if self.__workingStack[-1] == toRemoveFromWorkingStack[-1]: 
                        toRemoveFromWorkingStack.pop()
                    self.__workingStack.pop()
                    
                if len(toRemoveFromWorkingStack) != 0: 
                    raise(Exception('Cannot Parse reduce'))
                
                self.__inputStack.insert(0, reducedProduction[0])
                self.__output.insert(0, str(reducedState))
            
        return self.__output

# Lexicl Analyzer

In [ ]:
def isPartOfOperator(char): 
    for op in operators: 
        if char in op: 
            return True 
    return False 

In [ ]:
def isEscapedQuote(line, index):
    return False if index == 0 else line[index -1] == '\\'

In [ ]:
def getStringToken(line, index):
    token = ''
    quoteCount = 0
    
    while index < len(line) and quoteCount < 2:
        if line[index] == '"' and not isEscapedQuote(line, index):
            quoteCount += 1 
        token += line[index]
        index += 1
        
    return token, index

In [ ]:
def getOperatorToken(line, index):
    token = ''
    
    while index < len(line) and isPartOfOperator(line[index]):
        token += line[index]
        index += 1 
        
    return token, index

In [ ]:
def tokenGenerator(line, separators): 
    token = ''
    index = 0
    
    while index < len(line):
        if line[index] == '"': 
            if token: 
                yield token
            token, index = getStringToken(line, index)
            yield token
            token = ''
            
        elif isPartOfOperator(line[index]):
            if token: 
                yield token
            token, index = getOperatorToken(line, index)
            yield token
            token = ''
        
        elif line[index] in separators:
            if token: 
                yield token
            token, index = line[index], index + 1
            yield token
            token = ''
            
        else:
            token += line[index]
            index += 1 
    if token: 
        yield token

### Our language specification

In [ ]:
separators = ['[', ']', '{', '}', '(', ')', ';', ' '] 
operators = ['+', '-', '*', '/', '<', '<=', '=', '>=', '>', '>>', '<<', '==', '&&', '||', '!', '&']
reservedWords = ['int', 'char', 'bool', 'float', 'array', 'struct', 'if', 'else', 'for', 'while', 'cout', 'true', 'false']

everything = separators + operators + reservedWords
codification = dict( [ [everything[i], i + 2] for i in range(len(everything))])
codification['identifier'] = 0
codification['constant'] = 1

### Generating PIF

In [ ]:
class SymbolTable():
    def __init__(self):
        self.__content = dict() 
        self.__count = -1
        
    def add(self, value):
        self.__count += 1
        self.__content[self.__count] = value
        return self.__count
    
    def __str__(self):
        return str(self.__content)

In [ ]:
class ProgramInternalForm: 
    def __init__(self):
        self.__content = []
        
    def add(self, code, id):
        self.__content.append((code, id))
        
    def getCodes(self): 
        return [ code[0] for code in self.__content]
    
    def __str__(self):
        return str(self.__content)


In [ ]:
def isIdentifier(token): 
    return re.match(r'^[a-zA-Z]([a-zA-Z]|[0-9]|_){,8}$', token) is not None

In [ ]:
def isConstant(token):
    return re.match('^(0|[\+\-]?[1-9][0-9]*)$|^\'.\'$|^\".*\"$', token) is not None

In [ ]:
identifierTable = SymbolTable()
constantsTable = SymbolTable()
pif = ProgramInternalForm()

fileName = 'program.txt'

with open(fileName, 'r') as file: 
    lineNo = 0
    for line in file:
        lineNo += 1 
        for token in tokenGenerator(line.strip('\n'), separators): 
            if token == ' ': 
                continue
            if token in separators + operators + reservedWords: 
                pif.add(codification[token], -1)
            elif isIdentifier(token): 
                id = identifierTable.add(token)
                pif.add(codification['identifier'], id)
            elif isConstant(token): 
                id = constantsTable.add(token)
                pif.add(codification['constant'], id)
            else:
                raise Exception('Unknown token ' + token + ' at line ' + str(lineNo)) 
                
print('Program Internal Form: \n', pif)
print('Identifier Table: \n', identifierTable) 
print('Constants Table: \n', constantsTable)

### Turning PIF into an input stack for Lr(0)

In [ ]:
inverseCodification = dict( [ [codification[key], key] for key in codification])

for code in pif.getCodes(): 
    print(code, ' : ', inverseCodification[code])
    
inputStack = [str(code) for code in pif.getCodes()]
print(input)

### Verifying symbol codification from language specification

In [ ]:
for key in codification:
    print(key, ' : ', codification[key])

### Verifying Lr(0) Table

In [ ]:
for index, line in enumerate(lr0.getTable()): 
    print(index, line)

### Verifying closure

In [ ]:
s0 = lr0.closure([ ('S1', ['.', g.S]) ])

for s in g.N + g.E: 
    print('goto( s0, ' + s + ') = ', lr0.goTo(s0, s))

### Verifying Cannonical Collection

In [ ]:
g = Grammar.fromFile("grammar.txt")
lr0 = Lr0Parser(g)

for index, state in enumerate(lr0.getCannonicalCollection()):
    print(index, state)

### Using Lr(0) Parser on the input stack generated by PIF

In [ ]:
g = Grammar.fromFile("my_grammar.txt")
g.P = [('S1', ['.', g.S])] + g.P
g.N += ['S1']
lr0 = Lr0Parser(g)

print(g)

print(lr0.parse(inputStack))